C# records only live in memory in a single process. If we want to take that idea of immutable records beyond the .NET runtime, we'll need some help. That's what Jinaga does.

We can store immutable records -- what Jinaga calls _facts_ -- in a local database. And we can share them with other computers. And we get the same semantics: two facts with the same type and fields are actually the same fact.

In [ ]:
#r "../Jinaga.Graphviz/bin/Debug/net5.0/Jinaga.UnitTest.dll"
using Jinaga;

[FactType("Jinaga.User")]
record User(string publicKey);

string publicKey = "--- PUBLIC KEY ---";
new User(publicKey)

In [ ]:
using Jinaga.UnitTest;

var j = JinagaTest.Create();

var firstUser = await j.Fact(new User(publicKey));
var secondUser = await j.Fact(new User(publicKey));

firstUser.Equals(secondUser)

In [ ]:
#r "../Jinaga.Graphviz/bin/Debug/net5.0/Jinaga.Graphviz.dll"
using Jinaga.Graphviz;

[FactType("Corporate.Company")]
record Company(string identifier);

[FactType("Corporate.Employee")]
record Employee(Company company, int employeeNumber);

var contoso = await j.Fact(new Company("Contoso"));
var jane = await j.Fact(new Employee(contoso, 1));
var bob = await j.Fact(new Employee(contoso, 2));

var employees = await j.Query(contoso, Given<Company>.Match((company, facts) =>
    facts.OfType<Employee>().Where(employee => employee.company == company)
));
Renderer.RenderFacts(employees)

In [ ]:
[FactType("Corporate.Employee.Name")]
record EmployeeName(Employee employee, string name, EmployeeName[] prior);

var initialJane = await j.Fact(new EmployeeName(jane, "Jane", new EmployeeName[0]));
var initialBob = await j.Fact(new EmployeeName(bob, "Bob", new EmployeeName[0]));
var secondBobMispeld = await j.Fact(new EmployeeName(bob, "Rober", new [] { initialBob }));
var secondBob = await j.Fact(new EmployeeName(bob, "Robert", new [] { initialBob }));
var correctedBob = await j.Fact(new EmployeeName(bob, "Robert", new [] {
    secondBobMispeld, secondBob
}));

var namesOfEmployees = Given<Company>.Match((company, facts) =>
    from employeeName in facts.OfType<EmployeeName>()
    where employeeName.employee.company == company
    where !(
        from next in facts.OfType<EmployeeName>()
        where next.prior.Contains(employeeName)
        select next
    ).Any()
    select employeeName
);
var names = await j.Query(contoso, namesOfEmployees);
Renderer.ListTypes(typeof(EmployeeName))